In [2]:
import numpy as np 
import matplotlib.pyplot as plt 

In [6]:
%%capture


data=np.loadtxt("../MD_MC/MDNVE_MCNVT/output_epot.dat")
nfin=70
plt.errorbar(data[:nfin,0],data[:nfin,2],yerr=data[:nfin,3])
plt.plot(data[:nfin,0],data[:nfin,1])
plt.grid()
plt.show()